# Overview

The aim of this project is to examine the relationships between major coronavirus events and their impact on various economies.

***I do not have a partner. I reached out to multiple people, however, it was already too late. 
   Somebody came to me and like my topic, however, he could not get out of his current situation. ***

Major events are outlined from the CoronaNet Project https://www.coronanet-project.org/download.html. This data set includes coronavirus events for 35+ countries from January 2020 - August 2020. This data set is updated monthly and is redownloaded as such.

The first step in this process is to analyze what is a "good" vs. "bad" event. I will take key words, phrases that are justifiably bad, "State of emergency", "implement lockdown", "lifts lockdown", or "imposes"

Two comparisons will be examined
    1: Country vs. Country
    2: Country vs. Benchmark Indices.
    
In country vs. country analyses, I will be looking at the effect on exchange rates. I will take major events in one economy and see how it shifts exchange rates before and after. I want to see if there is a devaluation in currency not due to a forcasted drop in interest rates, nor an influx due to a stable balance sheet, but due to immediate investor reaction to a governments handling of the crisis. This will account the economic outlook of a nation in reaction to a major event.

In country vs. benchmark, I will take each country's major events and analyze how their respective stock markets react. For instance, in the United States, I would analyze how the Dow Jones, S&P 500, Russell 2000, and VIX moves in relation to big events. I have compiled a csv file with each countries benchmark index and their https://finance.yahoo.com/ tickers. The CSV file contains 14 different countries indices. 

I have already written code that webscrapes historical data from https://finance.yahoo.com/. I will use this end of day and opening price data to see major shifts in economic outlook resulting from a major event. I will use the requests library to scrape this data.
    - https://finance.yahoo.com/ loves to use excessive JSON's in the back of their html pages and can be used as such.
   
Steps:

1: Clean Data
2: Determine good vs. bad events in the description
3: Determine gravity of each event (coronvirus shutdown in 1 state -> combine events into "multiple states shut down")
4: Make boolean table on when each country has major events -> used to determine coincidences/possible biased results.
5: Take average price levels for indices and foreign exchange rates, and look for immediate/delayed price movement.
    - Put emphasis on countries not experiencing major events and have had their currencies "normalize".
6. Construct meta-data on the results and construct model to analyze data.

Organizational Structure:

I plan on pushing my updated code to a github repository so I can work on my project wherever I may be. I do not use the same computer to do all my work and need the flexibility to store my code/data online. If calculations get too large, or I need extra storage space, I plan on setting up an AWS S3 bucket to store my data, or I will use the google drive API to store data in google sheets. I do not anticipate this being the case. 




In [1]:
import requests
import csv
import time
import pandas as pd
import logging
import threading
import time
import json
import os


"""
import gspread
from pprint import pprint
from googleapiclient import discovery
"""

historical_url = "https://finance.yahoo.com/quote/%s/history?p=%s"


"""Adjusted to automatically take in symbols for automation"""
def generateUrl(symbol=False):
    url_1 = "https://finance.yahoo.com/quote/"
    url_2 = "?p="
    url_3 = "&.tsrc=fin-srch"
   
    if symbol == False:
        # Allows user to input stock symbol and returns the Yahoo finance URL
        symbol = input("Enter a ticker symbol for a stock: ")
        url = url_1 + symbol + url_2 + symbol + url_3
    else:
        url = url_1 + symbol + url_2 + symbol + url_3
       
    return url
       
"""Adjusted to allow symbols and different URL's to pass through
    Allows us to grab from historical, options, and other data sources"""
def dataGrabber(symbol=False, url=None):
    if url == None:
        # Obtain server's response to HTTP request
        url = generateUrl(symbol)
       
    src = requests.get(url)
    # Converts source code to string
    data = src.text
    return data
       


def getCurrentPrice(symbol=False):
    # Isolates the currentPrice variable
    data = dataGrabber(symbol)
    split = data.split("currentPrice\":", 1)
    halfList = split[1]
    split2 = halfList.split("}", 1)
    currentPrice = split2[0] + "}"
    finalPrice = currentPrice.split("""{"raw":""")[1].split(",")[0]
    # Print and return current price
    print(symbol, finalPrice)
    return round(float(finalPrice),3)


def getCSV(symbol=None):
    # Gets CSV of historical data on stock for the past 5 years
    url_1 = "https://query1.finance.yahoo.com/v7/finance/download/"
    url_2 = "?period1=1433980800&period2=" + t
    url_3 = "&interval=1d&events=history"
    if symbol == None:
        symbol = input("Enter a ticker symbol for a stock: ")
    url = url_1 + symbol + url_2 + url_3
    print(url)
    with requests.Session() as s:
        download = s.get(url)
        decoded_content = download.content.decode('utf-8')
        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        historical_data = list(cr)
        # Prints CSV
        c = True
       
        data = []
       
        for row in historical_data:
            if c == True:
                columns = row
                c=False
            else:
                data.append(row)
                print(row)
        
        df = pd.DataFrame(data=data, columns=columns)
        print(df)
       
        df.to_csv(symbol + ".csv")
       
        # Outputs CSV file to computer
        # Filename will be ticker symbol
    return

def getJSON(symbol, url = historical_url):
    url = url % (symbol, symbol)
   
    data = dataGrabber(url)
    start = '''"HistoricalPriceStore":{"prices":'''
    end = """]}"""
    print(url)
    print(data)
   
#getCSV('TSLA')
   
   
   
def priceToJSON(symbol):
    import os
   
    price = getCurrentPrice(symbol)
   
    if os.path.exists(symbol + '_current.txt'):
        os.remove(symbol + '_current.txt')
   
   
    with open(symbol + '_current.txt', 'x') as inFile:
        #df = json.loads(inFile.read())
        inFile.write(str(price))
   
    return

In [20]:
corona = pd.read_csv("data/coronanet_release.csv")
indices = pd.read_csv("data/Stock Market Indices by Country - Sheet1.csv")

print(corona.iloc[643]['description'])

Bangladesh imposes restriction on presence of state employees in the offices. Only 25% of the officials are allowed in the offices. Announced on June 01. On August 03, the state minister of Public Administration Ministry has announced to put an end to this requirement. The Orders have been given orally. The full attendance will be ensured from Sunday, August 09.
